# Loading and Processing TEI-Encoded XML
Because I wasn't able to get everyone working with the lxml library, I've simplified the process of loading xml and then processing it. Follow along below and let me know if you have any problems planning and implementing your assignment.

## Step 1: Read XML from github


In [1]:
# Imports

import urllib.request

In [2]:
# Variables

url_to_load = "https://raw.githubusercontent.com/EanaKim/dh-course-fall-2017/master/Art_Tech_Practice.xml" # url here
print(url_to_load)

https://raw.githubusercontent.com/EanaKim/dh-course-fall-2017/master/Art_Tech_Practice.xml


In [3]:
# assign your url to the variable url_to_load
# you can try http://papyri.info/ddbdp/bgu;1;133/source .

with urllib.request.urlopen(url_to_load) as f:
    tei_as_string = f.read().decode('utf-8')

print(tei_as_string)
# if the above was successful you should see a TEI document below:

<?xml version="1.0" encoding="UTF-8"?>
<?xml-model href="http://www.stoa.org/epidoc/schema/latest/tei-epidoc.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        <fileDesc>
           <titleStmt>
                <title>Art in the Information Age</title>
            </titleStmt>
            <publicationStmt>
                <publisher>Edward A. Shanken</publisher>
                <date when ="2002">2002</date>
            </publicationStmt>
            <sourceDesc>
                <p>Technology and Conceptual Art</p>
            </sourceDesc>
        </fileDesc>
    </teiHeader>
    <text>
        <body>
            <ab><date>In the mid-1960s</date>, <persName>Marshall McLuhan</persName> prophesied
            that electronic media were creating an increasingly interconnected
            global village. Such pronouncements popularized the
            idea that the era of machine-age technology 

## Step 2: We can already do things
The find() method of strings will return the index of a substring. Try it:

In [4]:
with urllib.request.urlopen(url_to_load) as f:
    tei_as_string = f.read().decode('utf-8')

start_index = tei_as_string.find('<lb') # find the index of the first l element

# why is the '[start_index:]' range important here?
tmp_end_index = tei_as_string[start_index:].find('>')

# what happens if we don't add 1 at the end? try it
end_index = start_index + tmp_end_index + 1

print("The first lb element begins at index " + str(start_index))
print("The first lb element ends at index " + str(end_index))

The first lb element begins at index -1
The first lb element ends at index 0


## Quiz!
Print the first lb element by using the start_index and end_index.

In [8]:
 tei_as_string[start_index:end_index]

''

## More complex elements
lb elements are zero-lengh, which means they can take the form '&lt;lb n="#" />' . Note the '/>' at the end of the element. This is equivalent to '&lt;lb n="#">&lt;/lb>' but is easier to write.

Next we want to capture elements of the form '&lt;persName type="a type">Tom&lt;/persName>'. You know all the python code to do this. We just put things together a little differently.

## Quiz
The cell below is almost ready to load the sample URL from the first cell, then to find and print out the first 'expan' element. Assign a value to end_tag to make the cell work.


In [9]:
#url_to_load = "http://papyri.info/ddbdp/bgu;1;133/source"

#with urllib.request.urlopen(url_to_load) as f:
#    tei_as_string = f.read().decode('utf-8')

# start_tag
start_tag = '<expan' # why leave off the '>'?
end_tag =  '>'

start_index = tei_as_string.find(start_tag)

tmp_end_index = tei_as_string[start_index:].find(end_tag)

end_index = start_index + tmp_end_index + len(end_tag) # why '+ len(end_tag)'?

print(tei_as_string[start_index:end_index])

## Quiz!
Cut-and-paste the working code from the above cell and adapt it so that it finds the first supplied element. Be careful, supplied elements can have the form '&lt;supplied reason="..."> ... &lt;/supplied>' This means you can just search for '&lt;supplied>'.

In [ ]:
### Your answer here

## A Better Way
Building your own indexes into a string to find start and end tags could get very complicated very quickly. Let's use a libray. In class we used the 'lxml' library. I recommend sticking with that if you're going to process xml in python as part of your final project. Now we'll use the 'xml' library, which should work on everyone's machine. In the following cells, look at the comments to follow along.

In [10]:
# import what we need from the xml library
import xml.etree.ElementTree as ET

In [11]:
# some simple xml as a test
xml_str = "<doc>\n\t<p>Hello World!</p>\n\t<p>Goodbye!!!</p></doc>"
print(xml_str)

<doc>
	<p>Hello World!</p>
	<p>Goodbye!!!</p></doc>


In [12]:
# turn this string into an ElementTree. Think of that as a very specialized version of a python list.
xml_elements = ET.fromstring(xml_str)

print(type(xml_str))
print(type(xml_elements))


<class 'str'>
<class 'xml.etree.ElementTree.Element'>


In [13]:
# ElementTrees have a .findall method that can take an xpath expression.
# IMPORTANT: Begin your xpath with '.'
# You can iterate over them with a for loop.

# for item in list:
#      do stuff


xml_elements.findall('p')


for p in xml_elements.findall('p'):
    print(p.text)

Hello World!
Goodbye!!!


In [25]:
# now let's access attributes
# we don't need to import the xml library again

# some simple xml, but now with attributes
xml_str = '<doc><p n="1" type="salutation">Hello World!</p><p n="2" type="farewell">Goodbye!!!</p></doc>'

# turn this string into an ElementTree. Think of that as a very specialized version of a python list.
xml_elements = ET.fromstring(xml_str)

# ElementTrees have a .findall method that can take an xpath expression.
# IMPORTANT: Begin your xpath with '.'
# You can iterate over them with a for loop.
#for p in xml_elements.findall('.//p'):
#    print(p.attrib['type']) # p.attrib on its own returns a dictionary.
 
ps = [element for element in xml_elements.findall('.//p')]
print(type(ps))
print(ps)

ps_text = [element.text for element in ps]
print(ps_text)

ps_items = [(element.attrib['n'], element.text) for element in ps]
print(ps_items)

<class 'list'>
[<Element 'p' at 0x108502a98>, <Element 'p' at 0x108502ae8>]
['Hello World!', 'Goodbye!!!']
[('1', 'Hello World!'), ('2', 'Goodbye!!!')]


In [14]:
for p in xml_elements.findall('.//p'):
    print(p.text) # p.attrib on its own returns a dictionary.

print('\n')

for p in xml_elements.findall('.//p[@n]'):
    print(p.text) # p.attrib on its own returns a dictionary.

print('\n')

for p in xml_elements.findall('.//p[@n="1"]'):
    print(p.text) # p.attrib on its own returns a dictionary.

Hello World!
Goodbye!!!






## Quiz!
Change the above code so that it prints out the n attribute of each p element.

In [ ]:
### Your code here

## An example with real TEI

In [15]:
# I suggest running this first 'as is'. Then try using a URL for your own TEI.

#url_to_load = "http://papyri.info/ddbdp/bgu;1;133/source"

#with urllib.request.urlopen(url_to_load) as f:
#    tei_as_string = f.read().decode('utf-8')

xml_elements = ET.fromstring(tei_as_string)

# IMPORTANT: In the findall(...) statements that follow,
# you'll see '{http://www.tei-c.org/ns/1.0}'. Leave it.
# But note that you can change the element name.

# if you are using your own TEI, you might need to change the XPATH so that is meaningful for your data

print("\nList found elements:")
# element.tag returns the name of the current element
elements = xml_elements.findall('.//{http://www.tei-c.org/ns/1.0}lb')
for element in elements:
    print(element.tag)

print("\nList the text of found elements:")
# element.text returns the text content of the current element. Here 'expan' elements are found.
elements = xml_elements.findall('.//{http://www.tei-c.org/ns/1.0}expan')
for element in elements:
    print(element.text)

print("\nList the value attributes of found elements")
# as you've seen element.attrib, return the attributes
elements = xml_elements.findall('.//{http://www.tei-c.org/ns/1.0}num')
for element in elements:
    print(element.attrib['value'])


List found elements:

List the text of found elements:

List the value attributes of found elements


In [16]:
from collections import Counter

expans = [expan.text for expan in xml_elements.findall('.//{http://www.tei-c.org/ns/1.0}expan')]

c = Counter(expans)
print(c)

Counter()


## A little bit of bad news
The xml library is simpler than the lxml library. It does not support xpath queries of the form .//num/@value . You have to get all the nums and move through them looking for value attributes using .attrib .

## Assignment!
You can now fetch your TEI from github, find certain elements within it, and then write a for loop to access those elements individually. The last step is like the 'for color in colors:' loop in Chapter 1.

Your assignment is to adapt the code above to do something interesting with your TEI-encoded xml file that is in the github repository. You could find shared words in paragraphs. List all rulers, dates, places, (organs?). How about making links to the wikipedia articles for rulers - or other proper nouns - mentioned in a text? How would you do that?

And as I mentioned, you might have to make changes to your TEI so that python can easily work with the elements within your document. That is OK. Just make sure the file remains valid XML.


In [26]:
url_to_load = "https://raw.githubusercontent.com/EanaKim/dh-course-fall-2017/master/Art_Tech_Practice.xml" 

with urllib.request.urlopen(url_to_load) as f:
    tei_as_string = f.read().decode('utf-8')
    
# import what we need from the xml library
import xml.etree.ElementTree as ET

xml_elements = ET.fromstring(tei_as_string)
for p in xml_elements.findall('.//{http://www.tei-c.org/ns/1.0}persName'):
    print(p.text)

#print(tei_as_string)

Marshall McLuhan
Pontus Hultén 
Robert Rauschenberg
Billy Klüver
Jack Burnham
Les Levine
Hans Haacke
Joseph Kosuth
Ned Woodman
Ted
                    Nelson)
Nicholas Negroponte


## How to turn in your assignment

First, exectute the cell with your code in it and make sure the output is what you want.

Then choose 'Save and checkpoint' from the "File" menu of this notebook.

Copy this file, which is 'tei-processing.ipynb' into your folder of the class repository. As usual, commit and sync changes to github. That's it. When loaded from github, the notebook will appear essentially the same as it does to you now.